In [70]:
import findspark
findspark.init()

In [71]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [72]:
#obj1 = spark.read.csv("C:\data\FINAL-2\objetivo1-final.csv",header=True, inferSchema=True, nullValue="NA")
obj1=spark.read.format("csv") \
    .option("quote", "") \
    .option("escape", "\"") \
    .option("header", "true") \
    .option("multiLine", "true") \
    .option("encoding","windows-1252") \
    .load("C:\data\FINAL-3\objetivo1-final.csv")

In [73]:
obj1.show(10)

+---+-------+---------------+---------------+------+-----+------+-----------+-------------------+-----------------+----+----------+--------------------+----------------------+
| ID|Nombres|ApellidoPaterno|ApellidoMaterno|  Peso|Talla|  Zona|NivelFiebre|AntecedentesFamilia|AdiccionDepresora|Anio|       Mes| CategoriaEnfermedad|SubcategoriaEnfermedad|
+---+-------+---------------+---------------+------+-----+------+-----------+-------------------+-----------------+----+----------+--------------------+----------------------+
|  1| Elston|        Danihel|         Grioli| 57.82| 1.34|URBANA|         38|                 SI|          NINGUNA|2019|   OCTUBRE|ENFERMEDAD RESPIR...|  J42 - BRONQUITIS ...|
|  2|  Abdul|     Grahamslaw|          Lukas| 89.09| 1.85| RURAL|         41|                 SI|       TABAQUISMO|2021|    AGOSTO|ENFERMEDAD RESPIR...|  J91 - DERRAME PLE...|
|  3|  Morse|       Dreghorn|      Monkhouse|105.99| 1.58|URBANA|         40|                 NO|       TABAQUISMO|2021|

# Selección de datos

In [74]:

#Objetivo 1 - Eliminando columnas que no son deseadas
obj1 = obj1.drop('ID','ApellidoPaterno','ApellidoMaterno','Nombres','CategoriaEnfermedad', 'Peso', 'Talla','Departamento','Zona','Mes','Anio')
obj1.show(10)
print(obj1.count())

obj1.printSchema()

+-----------+-------------------+-----------------+----------------------+
|NivelFiebre|AntecedentesFamilia|AdiccionDepresora|SubcategoriaEnfermedad|
+-----------+-------------------+-----------------+----------------------+
|         38|                 SI|          NINGUNA|  J42 - BRONQUITIS ...|
|         41|                 SI|       TABAQUISMO|  J91 - DERRAME PLE...|
|         40|                 NO|       TABAQUISMO|  J14 - NEUMONIA DE...|
|         41|                 SI|          NINGUNA|  J67 - NEUMONITIS ...|
|         36|                 NO|       TABAQUISMO|  J37 - LARINGITIS ...|
|         38|                 SI|DROGA ALUCINOGENA|  J01 - SINUSITIS A...|
|         41|                 NO|DROGA ALUCINOGENA|  J32 - SINUSITIS C...|
|         39|                 SI|          ALCOHOL|  J96 - INSUFICIENC...|
|         40|                 SI|          NINGUNA|  J62 - NEUMOCONIOS...|
|         41|                 SI|       TABAQUISMO|            J45 - ASMA|
+-----------+------------

# Limpieza de datos

In [75]:
#Eliminando valores que no corresponden del dataset
obj1 = obj1.dropna()
obj1 = obj1.filter('NivelFiebre < 45')
obj1 = obj1.where((obj1.AdiccionDepresora == 'NINGUNA') | (obj1.AdiccionDepresora == 'TABAQUISMO') | (obj1.AdiccionDepresora == 'DROGA ALUCINOGENA'))
obj1.show(10)
obj1.count()

+-----------+-------------------+-----------------+----------------------+
|NivelFiebre|AntecedentesFamilia|AdiccionDepresora|SubcategoriaEnfermedad|
+-----------+-------------------+-----------------+----------------------+
|         38|                 SI|          NINGUNA|  J42 - BRONQUITIS ...|
|         41|                 SI|       TABAQUISMO|  J91 - DERRAME PLE...|
|         40|                 NO|       TABAQUISMO|  J14 - NEUMONIA DE...|
|         41|                 SI|          NINGUNA|  J67 - NEUMONITIS ...|
|         36|                 NO|       TABAQUISMO|  J37 - LARINGITIS ...|
|         38|                 SI|DROGA ALUCINOGENA|  J01 - SINUSITIS A...|
|         41|                 NO|DROGA ALUCINOGENA|  J32 - SINUSITIS C...|
|         40|                 SI|          NINGUNA|  J62 - NEUMOCONIOS...|
|         41|                 SI|       TABAQUISMO|            J45 - ASMA|
|         39|                 SI|          NINGUNA|  J67 - NEUMONITIS ...|
+-----------+------------

221821

# Integracion de datos

In [77]:
from pyspark.ml.feature import StringIndexer

obj1 = StringIndexer(inputCol='NivelFiebre', outputCol= 'fiebre_idx').fit(obj1).transform(obj1)
obj1 = StringIndexer(inputCol='AntecedentesFamilia', outputCol= 'ant_idx').fit(obj1).transform(obj1)
obj1 = StringIndexer(inputCol='AdiccionDepresora', outputCol= 'adc_idx').fit(obj1).transform(obj1)

In [78]:
obj1.show(10)

+-----------+-------------------+-----------------+----------------------+----------+-------+-------+
|NivelFiebre|AntecedentesFamilia|AdiccionDepresora|SubcategoriaEnfermedad|fiebre_idx|ant_idx|adc_idx|
+-----------+-------------------+-----------------+----------------------+----------+-------+-------+
|         38|                 SI|          NINGUNA|  J42 - BRONQUITIS ...|       0.0|    1.0|    2.0|
|         41|                 SI|       TABAQUISMO|  J91 - DERRAME PLE...|       4.0|    1.0|    0.0|
|         40|                 NO|       TABAQUISMO|  J14 - NEUMONIA DE...|       2.0|    0.0|    0.0|
|         41|                 SI|          NINGUNA|  J67 - NEUMONITIS ...|       4.0|    1.0|    2.0|
|         36|                 NO|       TABAQUISMO|  J37 - LARINGITIS ...|       5.0|    0.0|    0.0|
|         38|                 SI|DROGA ALUCINOGENA|  J01 - SINUSITIS A...|       0.0|    1.0|    1.0|
|         41|                 NO|DROGA ALUCINOGENA|  J32 - SINUSITIS C...|       4

In [83]:
obj1 = StringIndexer(inputCol='SubcategoriaEnfermedad', outputCol= 'label').fit(obj1).transform(obj1)
obj1.show(10)

+-----------+-------------------+-----------------+----------------------+----------+-------+-------+-------------+-----+
|NivelFiebre|AntecedentesFamilia|AdiccionDepresora|SubcategoriaEnfermedad|fiebre_idx|ant_idx|adc_idx|     features|label|
+-----------+-------------------+-----------------+----------------------+----------+-------+-------+-------------+-----+
|         38|                 SI|          NINGUNA|  J42 - BRONQUITIS ...|       0.0|    1.0|    2.0|[0.0,1.0,2.0]| 72.0|
|         41|                 SI|       TABAQUISMO|  J91 - DERRAME PLE...|       4.0|    1.0|    0.0|[4.0,1.0,0.0]| 50.0|
|         40|                 NO|       TABAQUISMO|  J14 - NEUMONIA DE...|       2.0|    0.0|    0.0|[2.0,0.0,0.0]| 47.0|
|         41|                 SI|          NINGUNA|  J67 - NEUMONITIS ...|       4.0|    1.0|    2.0|[4.0,1.0,2.0]| 54.0|
|         36|                 NO|       TABAQUISMO|  J37 - LARINGITIS ...|       5.0|    0.0|    0.0|[5.0,0.0,0.0]| 27.0|
|         38|           

In [79]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['fiebre_idx','ant_idx','adc_idx'],outputCol='features' )
obj1 = assembler.transform(obj1)

In [80]:
obj1.show(10)

+-----------+-------------------+-----------------+----------------------+----------+-------+-------+-------------+
|NivelFiebre|AntecedentesFamilia|AdiccionDepresora|SubcategoriaEnfermedad|fiebre_idx|ant_idx|adc_idx|     features|
+-----------+-------------------+-----------------+----------------------+----------+-------+-------+-------------+
|         38|                 SI|          NINGUNA|  J42 - BRONQUITIS ...|       0.0|    1.0|    2.0|[0.0,1.0,2.0]|
|         41|                 SI|       TABAQUISMO|  J91 - DERRAME PLE...|       4.0|    1.0|    0.0|[4.0,1.0,0.0]|
|         40|                 NO|       TABAQUISMO|  J14 - NEUMONIA DE...|       2.0|    0.0|    0.0|[2.0,0.0,0.0]|
|         41|                 SI|          NINGUNA|  J67 - NEUMONITIS ...|       4.0|    1.0|    2.0|[4.0,1.0,2.0]|
|         36|                 NO|       TABAQUISMO|  J37 - LARINGITIS ...|       5.0|    0.0|    0.0|[5.0,0.0,0.0]|
|         38|                 SI|DROGA ALUCINOGENA|  J01 - SINUSITIS A..

In [84]:
obj1_train, obj1_test = obj1.randomSplit([0.8, 0.2], seed=23)
[obj1_train.count(), obj1_test.count()]

[176911, 44910]

# Aplicación del modelo de algoritmo

In [85]:
from pyspark.ml.classification import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree_model = tree.fit(obj1_train)

In [86]:
prediction = tree_model.transform(obj1_test)
prediction.select('label', 'prediction', 'probability').show()
prediction.groupBy("label", "prediction").count().show()
prediction.show()

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
| 13.0|       0.0|[0.05068650338305...|
+-----+----------+--------------------+
only showing top 20 rows

+-----+-------

In [38]:
from pyspark.sql.functions import round
#Exportar datos pronosticados
#prediction = prediction.withColumn('probability',round(prediction.probability,2))
df_prediction=prediction.drop('features','rawPrediction','probability')
df_prediction.coalesce(1).write.option('header',True).csv('predicion-obj1.csv')


# Evaluación del modelo de algoritmo

In [87]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

df_accuracy = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy").evaluate(prediction)
df_accuracy

0.05036740146960588

In [88]:
df_precision = MulticlassClassificationEvaluator(labelCol="label", metricName="weightedPrecision").evaluate(prediction)
df_precision

0.0025368751308004565

In [89]:
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()


In [90]:
precision = (TN+ TP)/(TN+TP+FN+FP)
print(precision)


0.05036740146960588
